In [29]:
from pathlib import Path
from sys import stdout
import numpy as np
import pandas as pd

from scipy.constants import Boltzmann, electron_volt
from pymatgen import units

import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}

In [106]:
#read in energies
#Calculated energies from QE for the different patterns are tabulated and read in
qe_energies1 = pd.read_csv('data/energy.out.collinear.csv', header=None)
qe_energies2 = pd.read_csv('data/energy.out.1x1x4.csv', header=None)
  

#combine all energies
qe_energies =  qe_energies2 #pd.concat([qe_energies1, qe_energies2])
#qe_energies_all = qe_energies_all.reset_index(drop=True).drop_duplicates(["pattern"], keep="first") 
qe_energies.columns = ["pattern", "num_sites", "total_energy" ]
qe_energies.head()


,pattern,num_sites,total_energy
0,nm,24,-14691.879405
1,z-1,24,-14694.011241
2,z-2,24,-14694.009371
3,z-3,24,-14694.011054
4,z-4,24,-14694.004233


In [107]:
exchange_model = pd.read_csv("data/exchange_model.csv")
exchange_model = exchange_model.drop(['Unnamed: 0'], axis=1)
exchange_model.tail()

,pattern,J1,J2,J3,J4,J5,J6,J7,J8
32,z-5,0.000000,0.000000,1.000000,-0.666667,0.000000,0.333333,1.000000,0.666667
33,z-6,0.333333,0.000000,1.000000,0.000000,0.666667,0.000000,1.000000,0.666667
34,z-7,0.000000,0.000000,0.333333,-0.666667,0.000000,-0.333333,0.333333,0.666667
35,z-8,0.000000,0.000000,0.666667,-1.333333,0.000000,0.000000,0.666667,0.666667
36,z-9,0.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.666667,0.666667


In [108]:
model_matrix = pd.merge(qe_energies, exchange_model,  on =["pattern"])
model_matrix = model_matrix.drop_duplicates(["pattern"]).reset_index(drop=True)


    
nm_energy = np.float64(model_matrix.query("pattern == 'nm'").loc[:, "total_energy"])
model_matrix["total_energy"] = ( (model_matrix["total_energy"] \
                                 - np.float64(model_matrix.query("pattern == 'nm'").loc[:, "total_energy"]))\
                                 / model_matrix["num_sites"] )\
                                 * 1000 * units.Ry_to_eV
    

model_matrix = model_matrix[model_matrix.pattern != "nm"].drop("num_sites", axis=1)

model_matrix

,pattern,total_energy,J1,J2,J3,J4,J5,J6,J7,J8
1,z-1,-1208.545778,1.333333,1.333333,2.666667,1.333333,2.666667,0.666667,2.666667,0.666667
2,z-2,-1207.485776,0.666667,0.666667,1.666667,0.666667,1.333333,0.333333,1.666667,0.666667
3,z-3,-1208.440005,0.333333,0.000000,1.000000,0.000000,0.666667,0.000000,1.000000,0.666667
4,z-4,-1204.573256,0.000000,0.666667,1.333333,0.000000,0.000000,0.666667,1.333333,0.666667
5,z-5,-1208.326749,0.000000,0.000000,1.000000,-0.666667,0.000000,0.333333,1.000000,0.666667
6,z-6,-1206.058471,0.333333,0.000000,1.000000,0.000000,0.666667,0.000000,1.000000,0.666667
7,z-7,-1205.738708,0.000000,0.000000,0.333333,-0.666667,0.000000,-0.333333,0.333333,0.666667
8,z-8,-1208.519423,0.000000,0.000000,0.666667,-1.333333,0.000000,0.000000,0.666667,0.666667
9,z-9,-1206.182793,0.000000,0.666667,0.666667,0.000000,0.000000,0.000000,0.666667,0.666667
10,z-10,-1207.408938,0.000000,0.000000,0.333333,-0.666667,0.000000,-0.333333,0.333333,0.666667


In [109]:
smf_exchange_parameters= pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"])
for i in range(10, len(model_matrix)):
    smf_exchange_fit = smf.ols(data=model_matrix[:i], formula="total_energy ~ J1 + J2 + J3 + J4 +J5 + J6 + J7 + J8 ").fit()
    #smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["stats_models"])
    smf_exchange_parameters["model_"+str(i)] = pd.DataFrame(smf_exchange_fit.params)
    
smf_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17
Intercept,-706.517526,-706.651556,-706.823420,-706.776561,-707.036723,-706.839017,-706.575671,-706.494419
J1,54.629492,54.726588,54.781892,54.813801,54.638371,54.594252,54.586734,54.599754
J2,0.738973,0.761426,1.258452,1.495013,0.489773,0.065769,-1.529433,-1.618192
J3,-138.939663,-138.937913,-138.976079,-139.047401,-138.533165,-138.525922,-138.381506,-138.448925
J4,2.070343,1.789680,1.479039,1.378219,0.891812,1.195682,1.788898,2.113817
J5,109.258984,109.453176,109.563785,109.627601,109.276743,109.188503,109.173468,109.199508
J6,277.442529,277.436536,277.457642,277.323172,278.186279,278.105838,278.082208,277.947601
J7,-138.939663,-138.937913,-138.976079,-139.047401,-138.533165,-138.525922,-138.381506,-138.448925
J8,-471.011684,-471.101037,-471.215613,-471.184374,-471.357815,-471.226011,-471.050447,-470.996279


In [110]:
lm = LinearRegression()
lm_exchange_parameters = pd.DataFrame(index=["Intercept", "J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"])

for i in range(10, len(model_matrix)):
    model_matrix_i = model_matrix[:i]
    lm_exchange_fit = lm.fit(X=model_matrix_i[["J1", "J2", "J3", "J4", "J5", "J6", "J7", "J8"]], 
                             y=model_matrix_i["total_energy"])
    parameters = [lm_exchange_fit.intercept_, 
                lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1],lm_exchange_fit.coef_[2],
                lm_exchange_fit.coef_[3], lm_exchange_fit.coef_[4],lm_exchange_fit.coef_[5],
                lm_exchange_fit.coef_[6], lm_exchange_fit.coef_[7]]
    
    lm_exchange_parameters ["model_"+str(i)]= parameters
 
 
lm_exchange_parameters

,model_10,model_11,model_12,model_13,model_14,model_15,model_16,model_17
Intercept,-1.205236e+03,-1.205464e+03,-1205.757599,-1205.677663,-1206.121469,-1.205784e+03,-1205.334969,-1205.196361
J1,-7.836471e-01,-6.970635e-01,-0.655239,-0.619655,-0.815489,-8.441026e-01,-0.830966,-0.811573
J2,7.389726e-01,7.614256e-01,1.258452,1.495013,0.489773,6.576916e-02,-1.529433,-1.618192
J3,-4.068144e-01,-3.787848e-01,-0.383252,-0.463762,0.101487,6.996367e-02,0.162743,0.079393
J4,2.070343e+00,1.789680e+00,1.479039,1.378219,0.891812,1.195682e+00,1.788898,2.113817
J5,-1.567294e+00,-1.394127e+00,-1.310477,-1.239310,-1.630978,-1.688205e+00,-1.661931,-1.623146
J6,3.768327e-01,3.182787e-01,0.271987,0.155893,0.916976,9.140662e-01,0.993709,0.890966
J7,-4.068144e-01,-3.787848e-01,-0.383252,-0.463762,0.101487,6.996367e-02,0.162743,0.079393
J8,-5.192679e-31,5.433730e-31,0.000000,0.000000,0.000000,-7.096407e-28,0.000000,0.000000
